In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
import glob

# One Account

In [2]:
#Loading ST mapping file
st_mapping = pd.read_excel(r"C:\Users\cmedinag\OneDrive - Nub78\Documentos\Tableau CM\Mapping Files\2-Success_Track_Pricing_List.xlsx",dtype={'L2NBD':float, 'L2SWT':float},sheet_name="Success Track PIDs").fillna(0)
st_mapping = st_mapping[['Product SKU','L2NBD','L2SWT']]
st_mapping

,Product SKU,L2NBD,L2SWT
0,6807-S6T-10G-40G,13885.0,0.0
1,ACI-AD-GF,0.0,0.0
2,ACI-AD-GF=,0.0,0.0
3,ACI-AD-XF,0.0,0.0
4,ACI-AD-XF=,0.0,0.0
...,...,...,...
5430,WS4510E-S8+96SFP++,13127.0,0.0
5431,WSC2960X24TSLL++RF,286.0,0.0
5432,WSC3560CX12TCS++RF,247.0,0.0
5433,WSC3560CX12TCS++WS,247.0,0.0


In [4]:
IB = pd.read_csv(r'C:\Users\cmedinag\OneDrive - Nub78\Documentos\Tableau CM\OP\March\2022-03-09\OP_EMEAR_GU ID_124431_CLARIANTDEUTSC_2022_3_9\Data\IB\IB.csv')
IB

,Best Site GU Party ID,Best Site GU Party Name,Best Site ID,Best Site Customer Name,Best Site Sales Level 2 Name,Coverage,Contract Number,Covered Line Status,Contract Type,Contract Line End Quarter,...,SWSS Flag,Default Service List Price USD,Item Quantity,Annual Extended Contract Line List USD Amount,Annual Contract Line Net USD Amount,Annualized Extended Contract Line List USD Amount,Annualized Contract Line Net USD Amount,Contract Line List Price USD,Contract Line Net Price USD,Asset List Amount
0,124431,CLARIANT AG,-9999.0,UNKNOWN,US COMMERCIAL,NOT COVERED,NaN,NaN,NaN,19009,...,N,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,0.00
1,124431,CLARIANT AG,-9999.0,UNKNOWN,LATIN AMERICA,NOT COVERED,NaN,NaN,NaN,19009,...,N,NaN,19.0,NaN,NaN,NaN,NaN,NaN,NaN,0.00
2,124431,CLARIANT AG,-9999.0,UNKNOWN,EMEAR-CENTRAL,NOT COVERED,NaN,NaN,NaN,19009,...,N,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.00
3,124431,CLARIANT AG,-9999.0,UNKNOWN,EMEAR-CENTRAL,NOT COVERED,NaN,NaN,NaN,19009,...,N,NaN,168.0,NaN,NaN,NaN,NaN,NaN,NaN,0.00
4,124431,CLARIANT AG,-9999.0,UNKNOWN,EMEAR-CENTRAL,NOT COVERED,NaN,NaN,NaN,19009,...,N,NaN,81.0,NaN,NaN,NaN,NaN,NaN,NaN,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3199,124431,CLARIANT AG,-9999.0,UNKNOWN,US COMMERCIAL,COVERED,202437026.0,ACTIVE,3SNT,20232,...,N,NaN,3.0,0.0,0.0,0.0,0.0,0.0,0.0,8025.00
3200,124431,CLARIANT AG,-9999.0,UNKNOWN,EMEAR-SOUTH,COVERED,204527546.0,ACTIVE,SNTP,20252,...,N,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,16478.00
3201,124431,CLARIANT AG,-9999.0,UNKNOWN,EMEAR-CENTRAL,NOT COVERED,NaN,NaN,NaN,19009,...,N,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,647.35
3202,124431,CLARIANT AG,-9999.0,UNKNOWN,EMEAR_GERMANY,COVERED,202679777.0,ACTIVE,SNTP,20224,...,N,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1714.90


In [13]:
IB.iloc[:,32]

0           0.00
1           0.00
2           0.00
3           0.00
4           0.00
          ...   
3199     8025.00
3200    16478.00
3201      647.35
3202     1714.90
3203     2675.00
Name: Asset List Amount, Length: 3204, dtype: float64

In [8]:

#------------------------------------------------Calculating Success Tracks Column-------------------------------------------------
#if product id is not on mapping, mark it as 'Not Elegible'
merge1 = IB.merge(st_mapping, left_on='Product ID', right_on='Product SKU', how='left')
merge1 = merge1.rename(columns={'Product SKU':'Success Tracks'})
merge1['Success Tracks'] = merge1['Success Tracks'].fillna('Not Elegible')
j=merge1.iloc[0,0]
#-----------------------------------------------------------------------------------------------------------------
# if contract type is in list_st mark it as 'Not Elegible'
lista_st = ['L1NB3','L1NB5','L1NBD','L1SWT','L24H3','L24H5','L24HR','L2NB3','L2NB5','L2NBD','L2SWT']
for i in merge1.index:
    if type(merge1.iloc[i,8]) == float:
        pass
    else:
        if merge1.iloc[i,8] in lista_st:
            merge1.iloc[i,33]=merge1.iloc[i,8].replace(merge1.iloc[i,8],'Not Elegible')
#------------------------------------------------------------------------------------------------------------------
#Else mark it as 'Elegible'
for i in merge1.index:
    if merge1.iloc[i,33] != 'Not Elegible':
        merge1.iloc[i,33]=merge1.iloc[i,33].replace(merge1.iloc[i,33],'Elegible')

#-----------------------------------------------Calculating SmartNet NBD ST Eligible-----------------------------------------------
#If the product it's elegible use the annualized extended contract line list USD Amount, otherwise it's 0
merge1['SmartNet NBD ST Eligible'] = merge1['Annualized Extended Contract Line List USD Amount']
for i in merge1.index:
    if merge1.iloc[i,33] != 'Elegible':
        merge1.iloc[i,36] = 0
#-----------------------------------------------Calculating DNA HW Appliance-----------------------------------------------
#Check if 'DN2-HW-APL' or 'DN1-HW-APL' is in product ID and mark it as 1
merge1['DNA HW Appliance'] = merge1['Product ID']
for i in merge1.index:
    if 'DN2-HW-APL' in merge1.iloc[i,21] or 'DN1-HW-APL' in merge1.iloc[i,21]:
        merge1.iloc[i,37] = merge1.iloc[i,37].replace(merge1.iloc[i,37], '1')
# If not mark it as 0
for i in merge1.index:
    if merge1.iloc[i,37] != '1':
        merge1.iloc[i,37] = '0'

#-----------------------------------------------Calculating SmartNet ST Not Covered-----------------------------------------------
# If it's not covered and 1 in DNA HW Appliance get the annualized extended contract line list USD Amount
merge1['SmartNet ST Not Covered'] = 0
for i in merge1.index:
    if merge1.iloc[i,5] == 'NOT COVERED' and merge1.iloc[i,37] == '1' :
        merge1.iloc[i,38] = merge1.iloc[i,28]

#-----------------------------------------------Calculating ST Estimated List Price-----------------------------------------------
#merge1['ST Estimated List Price'] = (merge1['L2NBD'] + merge1['L2SWT'])*merge1['Item Quantity']
merge1['ST Estimated List Price'] = merge1['L2NBD']*merge1['Item Quantity']

#------------------------------------------------------Filtering------------------------------------------------------------------
merge1 = merge1[merge1['Coverage'] == 'COVERED']
merge1 = merge1[merge1['Success Tracks'] == 'Elegible']

#----------------------------------------Calculating Total ST Oppty KPI------------------------------------------------------------

sn_nbd_st_e = merge1['SmartNet NBD ST Eligible'].sum()
sn_st_not_covered = merge1['SmartNet ST Not Covered'].sum()
st_e_list_price = merge1['ST Estimated List Price'].sum()
ST_Oppty= st_e_list_price - sn_st_not_covered - sn_nbd_st_e

if len(merge1) == 0:
    #lista_oppty.append('N/A')
    #lista_sav_id.append(j)
    print('Account ID: ',j,'ST_Oppty: ', 'N/A')


if ST_Oppty < 0:

    merge1 = IB.merge(st_mapping, left_on='Product ID', right_on='Product SKU', how='left')
    merge1 = merge1.rename(columns={'Product SKU':'Success Tracks'})
    merge1['Success Tracks'] = merge1['Success Tracks'].fillna('Not Elegible')
    #-----------------------------------------------------------------------------------------------------------------
    # if contract type is in list_st mark it as 'Not Elegible'
    lista_st = ['L1NB3','L1NB5','L1NBD','L1SWT','L24H3','L24H5','L24HR','L2NB3','L2NB5','L2NBD','L2SWT']
    for i in merge1.index:
        if type(merge1.iloc[i,8]) == float:
            pass
        else:
            if merge1.iloc[i,8] in lista_st:
                merge1.iloc[i,33]=merge1.iloc[i,8].replace(merge1.iloc[i,8],'Not Elegible')
    #------------------------------------------------------------------------------------------------------------------
    #Else mark it as 'Elegible'
    for i in merge1.index:
        if merge1.iloc[i,33] != 'Not Elegible':
            merge1.iloc[i,33]=merge1.iloc[i,33].replace(merge1.iloc[i,33],'Elegible')


    merge1['SmartNet NBD ST Eligible'] = merge1['Default Service List Price USD']
    for i in merge1.index:
        if merge1.iloc[i,33] != 'Elegible':
            merge1.iloc[i,36] = 0
    #-----------------------------------------------Calculating DNA HW Appliance-----------------------------------------------
    #Check if 'DN2-HW-APL' or 'DN1-HW-APL' is in product ID and mark it as 1
    merge1['DNA HW Appliance'] = merge1['Product ID']
    for i in merge1.index:
        if 'DN2-HW-APL' in merge1.iloc[i,21] or 'DN1-HW-APL' in merge1.iloc[i,21]:
            merge1.iloc[i,37] = merge1.iloc[i,37].replace(merge1.iloc[i,37], '1')
    # If not mark it as 0
    for i in merge1.index:
        if merge1.iloc[i,37] != '1':
            merge1.iloc[i,37] = '0'

    #-----------------------------------------------Calculating SmartNet ST Not Covered-----------------------------------------------
    # If it's not covered and 1 in DNA HW Appliance get the annualized extended contract line list USD Amount
    merge1['SmartNet ST Not Covered'] = 0
    for i in merge1.index:
        if merge1.iloc[i,5] == 'NOT COVERED' and merge1.iloc[i,37] == '1' :
            merge1.iloc[i,38] = merge1.iloc[i,24]

    #-----------------------------------------------Calculating ST Estimated List Price-----------------------------------------------
    #merge1['ST Estimated List Price'] = (merge1['L2NBD'] + merge1['L2SWT'])*merge1['Item Quantity']
    merge1['ST Estimated List Price'] = merge1['L2NBD']*merge1['Item Quantity']

    #------------------------------------------------------Filtering------------------------------------------------------------------
    merge1 = merge1[merge1['Coverage'] == 'COVERED']
    merge1 = merge1[merge1['Success Tracks'] == 'Elegible']

    #----------------------------------------Calculating Total ST Oppty KPI------------------------------------------------------------

    sn_nbd_st_e = merge1['SmartNet NBD ST Eligible'].sum()
    sn_st_not_covered = merge1['SmartNet ST Not Covered'].sum()
    st_e_list_price = merge1['ST Estimated List Price'].sum()
    ST_Oppty= st_e_list_price - sn_st_not_covered - sn_nbd_st_e
    #lista_oppty.append(round(ST_Oppty/1000, 1))
    #lista_sav_id.append(j)
    print('Account ID: ',j,'ST_Oppty: ', round(ST_Oppty/1000, 1), 'Estimated list price: ', st_e_list_price, 'Originally Negative')
else:
    #lista_oppty.append(round(ST_Oppty/1000, 1))
    #lista_sav_id.append(j)
    print('Account ID: ',j,'ST_Oppty: ', round(ST_Oppty/1000, 1), 'Estimated list price: ', st_e_list_price)




Account ID:  283715236 ST_Oppty:  4357.0 Estimated list price:  12636914.43


# FUNCTION

In [13]:
def ST_Oppty(path,st_mapping):
    '''
    Def:
        Calculates the Success Tracks opportunity.
    Inputs:
        df = dataframe containing the IB
        st_mapping = Success tracks price list
    Outs:
        ST_Oppty value in k
    '''
    #Loading necesary files
    st_mapping = st_mapping[['Product SKU','L2NBD','L2SWT']]
    IB = pd.read_csv(path)
    #IB = df.copy()
    
    #------------------------------------------------Calculating Success Tracks Column-------------------------------------------------
    #if product id is not on mapping, mark it as 'Not Elegible'
    merge1 = IB.merge(st_mapping, left_on='Product ID', right_on='Product SKU', how='left')
    merge1 = merge1.rename(columns={'Product SKU':'Success Tracks'})
    merge1['Success Tracks'] = merge1['Success Tracks'].fillna('Not Elegible')
    j=merge1.iloc[0,0]
    #-----------------------------------------------------------------------------------------------------------------
    # if contract type is in list_st mark it as 'Not Elegible'
    lista_st = ['L1NB3','L1NB5','L1NBD','L1SWT','L24H3','L24H5','L24HR','L2NB3','L2NB5','L2NBD','L2SWT']
    for i in merge1.index:
        if type(merge1.iloc[i,8]) == float:
            pass
        else:
            if merge1.iloc[i,8] in lista_st:
                merge1.iloc[i,33]=merge1.iloc[i,8].replace(merge1.iloc[i,8],'Not Elegible')
    #------------------------------------------------------------------------------------------------------------------
    #Else mark it as 'Elegible'
    for i in merge1.index:
        if merge1.iloc[i,33] != 'Not Elegible':
            merge1.iloc[i,33]=merge1.iloc[i,33].replace(merge1.iloc[i,33],'Elegible')

    #-----------------------------------------------Calculating SmartNet NBD ST Eligible-----------------------------------------------
    #If the product it's elegible use the annualized extended contract line list USD Amount, otherwise it's 0
    merge1['SmartNet NBD ST Eligible'] = merge1['Annualized Extended Contract Line List USD Amount']
    for i in merge1.index:
        if merge1.iloc[i,33] != 'Elegible':
            merge1.iloc[i,36] = 0
    #-----------------------------------------------Calculating DNA HW Appliance-----------------------------------------------
    #Check if 'DN2-HW-APL' or 'DN1-HW-APL' is in product ID and mark it as 1
    merge1['DNA HW Appliance'] = merge1['Product ID']
    for i in merge1.index:
        if 'DN2-HW-APL' in merge1.iloc[i,21] or 'DN1-HW-APL' in merge1.iloc[i,21]:
            merge1.iloc[i,37] = merge1.iloc[i,37].replace(merge1.iloc[i,37], '1')
    # If not mark it as 0
    for i in merge1.index:
        if merge1.iloc[i,37] != '1':
            merge1.iloc[i,37] = '0'

    #-----------------------------------------------Calculating SmartNet ST Not Covered-----------------------------------------------
    # If it's not covered and 1 in DNA HW Appliance get the annualized extended contract line list USD Amount
    merge1['SmartNet ST Not Covered'] = 0
    for i in merge1.index:
        if merge1.iloc[i,5] == 'NOT COVERED' and merge1.iloc[i,37] == '1' :
            merge1.iloc[i,38] = merge1.iloc[i,28]

    #-----------------------------------------------Calculating ST Estimated List Price-----------------------------------------------
    #merge1['ST Estimated List Price'] = (merge1['L2NBD'] + merge1['L2SWT'])*merge1['Item Quantity']
    merge1['ST Estimated List Price'] = merge1['L2NBD']*merge1['Item Quantity']

    #------------------------------------------------------Filtering------------------------------------------------------------------
    merge1 = merge1[merge1['Coverage'] == 'COVERED']
    merge1 = merge1[merge1['Success Tracks'] == 'Elegible']

    #----------------------------------------Calculating Total ST Oppty KPI------------------------------------------------------------

    sn_nbd_st_e = merge1['SmartNet NBD ST Eligible'].sum()
    sn_st_not_covered = merge1['SmartNet ST Not Covered'].sum()
    st_e_list_price = merge1['ST Estimated List Price'].sum()
    ST_Oppty= st_e_list_price - sn_st_not_covered - sn_nbd_st_e

    if len(merge1) == 0:
        lista_oppty.append('N/A')
        lista_sav_id.append(j)
        return 'Account ID: ',j,'ST_Oppty: ', 'N/A'


    if ST_Oppty < 0:

        merge1 = IB.merge(st_mapping, left_on='Product ID', right_on='Product SKU', how='left')
        merge1 = merge1.rename(columns={'Product SKU':'Success Tracks'})
        merge1['Success Tracks'] = merge1['Success Tracks'].fillna('Not Elegible')
        #-----------------------------------------------------------------------------------------------------------------
        # if contract type is in list_st mark it as 'Not Elegible'
        lista_st = ['L1NB3','L1NB5','L1NBD','L1SWT','L24H3','L24H5','L24HR','L2NB3','L2NB5','L2NBD','L2SWT']
        for i in merge1.index:
            if type(merge1.iloc[i,8]) == float:
                pass
            else:
                if merge1.iloc[i,8] in lista_st:
                    merge1.iloc[i,33]=merge1.iloc[i,8].replace(merge1.iloc[i,8],'Not Elegible')
        #------------------------------------------------------------------------------------------------------------------
        #Else mark it as 'Elegible'
        for i in merge1.index:
            if merge1.iloc[i,33] != 'Not Elegible':
                merge1.iloc[i,33]=merge1.iloc[i,33].replace(merge1.iloc[i,33],'Elegible')


        merge1['SmartNet NBD ST Eligible'] = merge1['Default Service List Price USD']
        for i in merge1.index:
            if merge1.iloc[i,33] != 'Elegible':
                merge1.iloc[i,36] = 0
        #-----------------------------------------------Calculating DNA HW Appliance-----------------------------------------------
        #Check if 'DN2-HW-APL' or 'DN1-HW-APL' is in product ID and mark it as 1
        merge1['DNA HW Appliance'] = merge1['Product ID']
        for i in merge1.index:
            if 'DN2-HW-APL' in merge1.iloc[i,21] or 'DN1-HW-APL' in merge1.iloc[i,21]:
                merge1.iloc[i,37] = merge1.iloc[i,37].replace(merge1.iloc[i,37], '1')
        # If not mark it as 0
        for i in merge1.index:
            if merge1.iloc[i,37] != '1':
                merge1.iloc[i,37] = '0'

        #-----------------------------------------------Calculating SmartNet ST Not Covered-----------------------------------------------
        # If it's not covered and 1 in DNA HW Appliance get the annualized extended contract line list USD Amount
        merge1['SmartNet ST Not Covered'] = 0
        for i in merge1.index:
            if merge1.iloc[i,5] == 'NOT COVERED' and merge1.iloc[i,37] == '1' :
                merge1.iloc[i,38] = merge1.iloc[i,24]

        #-----------------------------------------------Calculating ST Estimated List Price-----------------------------------------------
        #merge1['ST Estimated List Price'] = (merge1['L2NBD'] + merge1['L2SWT'])*merge1['Item Quantity']
        merge1['ST Estimated List Price'] = merge1['L2NBD']*merge1['Item Quantity']

        #------------------------------------------------------Filtering------------------------------------------------------------------
        merge1 = merge1[merge1['Coverage'] == 'COVERED']
        merge1 = merge1[merge1['Success Tracks'] == 'Elegible']

        #----------------------------------------Calculating Total ST Oppty KPI------------------------------------------------------------

        sn_nbd_st_e = merge1['SmartNet NBD ST Eligible'].sum()
        sn_st_not_covered = merge1['SmartNet ST Not Covered'].sum()
        st_e_list_price = merge1['ST Estimated List Price'].sum()
        ST_Oppty= st_e_list_price - sn_st_not_covered - sn_nbd_st_e
        lista_oppty.append(round(ST_Oppty/1000, 1))
        lista_sav_id.append(j)
        return 'Account ID: ',j,'ST_Oppty: ', round(ST_Oppty/1000, 1), 'Estimated list price: ', st_e_list_price, 'Originally Negative'
    else:
        lista_oppty.append(round(ST_Oppty/1000, 1))
        lista_sav_id.append(j)
        return 'Account ID: ',j,'ST_Oppty: ', round(ST_Oppty/1000, 1), 'Estimated list price: ', st_e_list_price


# TESTING

In [16]:
path = r'C:\Users\cmedinag\OneDrive - Nub78\Documentos\Tableau CM\OP\February\2022-02-04\*'


#El codigo extrae la url de cada imagen en el pc
listaarchivos = []
for file in glob.glob(path):
    print(file+'\Data\IB\IB.csv')
    listaarchivos.append(file+'\Data\IB\IB.csv')

C:\Users\cmedinag\OneDrive - Nub78\Documentos\Tableau CM\OP\February\2022-02-04\OP_APJC_SAV ID_203797997_Test13_2022_2_4\Data\IB\IB.csv
C:\Users\cmedinag\OneDrive - Nub78\Documentos\Tableau CM\OP\February\2022-02-04\OP_APJC_SAV ID_203892264_Test14_2022_2_4\Data\IB\IB.csv
C:\Users\cmedinag\OneDrive - Nub78\Documentos\Tableau CM\OP\February\2022-02-04\OP_APJC_SAV ID_283714970_Test11_2022_2_4\Data\IB\IB.csv
C:\Users\cmedinag\OneDrive - Nub78\Documentos\Tableau CM\OP\February\2022-02-04\OP_APJC_SAV ID_283715230_Test12_2022_2_4\Data\IB\IB.csv
C:\Users\cmedinag\OneDrive - Nub78\Documentos\Tableau CM\OP\February\2022-02-04\OP_APJC_SAV ID_283715236_Test10_2022_2_4\Data\IB\IB.csv


In [17]:
%%time
# Ahora se calculan todos lo IB value de los archivos mediante un ciclo
# Calculando todos los IB_Value en bache
#Las listas se generan afuera y en ellas se almacenan los datos calculados
lista_oppty = []
lista_sav_id = []
for i in listaarchivos:
    print(ST_Oppty(i, st_mapping))

('Account ID: ', 203797997, 'ST_Oppty: ', 22797.1, 'Estimated list price: ', 42845365.67)
('Account ID: ', 203892264, 'ST_Oppty: ', 482.1, 'Estimated list price: ', 973047.51)
('Account ID: ', 283714970, 'ST_Oppty: ', 2028.7, 'Estimated list price: ', 4668567.01)
('Account ID: ', 283715230, 'ST_Oppty: ', 4250.0, 'Estimated list price: ', 14936245.44)
('Account ID: ', 283715236, 'ST_Oppty: ', 4357.0, 'Estimated list price: ', 12636914.43)
Wall time: 12min 7s


# More than one Account in IB file

In [3]:
path = r'C:\Users\cmedinag\OneDrive - Nub78\Documentos\Initiatives\FY22Q2 - High Potential Acoounts\HPA DATA2\*'

lista_sav_IB= []

for file in glob.glob(path):
    lista_sav_IB.append(file)
    
SAV_IB =pd.DataFrame()

for i in lista_sav_IB:
    df =pd.read_csv(i,dtype={'SAV ID':str,
                            'Instance Shipped Fiscal Year':str,
                            'Item Quantity':float,
                            'LDoS FY':str,
                            'Best Site ID':int,
                            'Contract Type':str,
                            'Covered Line Status':str,
                            'Contract Number':str,
                            'Service Brand Code':str,
                            "Asset List Amount":float},decimal=",")
    df.insert(loc=2, column='Best Site ID', value=-9999)
    df.insert(loc=3, column='Best Site Customer Name', value='UNKNOWN')
    SAV_IB = pd.concat([df,SAV_IB], axis=0)
IB_full = SAV_IB
IB_full

,SAV ID,SAV Name,Best Site ID,Best Site Customer Name,Best Site Sales Level 2 Name,Coverage,Contract Number,Covered Line Status,Contract Type,Contract Line End Quarter,...,SWSS Flag,Default Service List Price USD,Item Quantity,Annual Extended Contract Line List USD Amount,Annual Contract Line Net USD Amount,Annualized Extended Contract Line List USD Amount,Annualized Contract Line Net USD Amount,Contract Line List Price USD,Contract Line Net Price USD,Asset List Amount
0,222173994,BELIZE TELECOM,-9999,UNKNOWN,LATIN AMERICA,NOT COVERED,NaN,NaN,NaN,19009,...,N,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,512.01
1,222173994,BELIZE TELECOM,-9999,UNKNOWN,LATIN AMERICA,NOT COVERED,NaN,NaN,NaN,19009,...,N,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,2589.44
2,222173994,BELIZE TELECOM,-9999,UNKNOWN,LATIN AMERICA,NOT COVERED,NaN,NaN,NaN,19009,...,N,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,229.53
3,222173994,BELIZE TELECOM,-9999,UNKNOWN,LATIN AMERICA,NOT COVERED,NaN,NaN,NaN,19009,...,N,NaN,74.0,NaN,NaN,NaN,NaN,NaN,NaN,13064.70
4,222173994,BELIZE TELECOM,-9999,UNKNOWN,LATIN AMERICA,NOT COVERED,NaN,NaN,NaN,19009,...,N,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,423.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415803,283715230,SBU_PSR_MEX_CENTER_PSR105,-9999,UNKNOWN,LATIN AMERICA,COVERED,204218335,ACTIVE,PSUP,20251,...,N,165.0,1.0,479.85,86.276667,159.95,86.276667,479.85,258.83,2818.92
415804,283715230,SBU_PSR_MEX_CENTER_PSR105,-9999,UNKNOWN,LATIN AMERICA,NOT COVERED,NaN,NaN,NaN,19009,...,N,3636.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,89863.95
415805,283715230,SBU_PSR_MEX_CENTER_PSR105,-9999,UNKNOWN,LATIN AMERICA,NOT COVERED,NaN,NaN,NaN,19009,...,N,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,42889.88
415806,283715230,SBU_PSR_MEX_CENTER_PSR105,-9999,UNKNOWN,LATIN AMERICA,NOT COVERED,NaN,NaN,NaN,19009,...,N,2882.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,60536.00


In [4]:
IB_full[IB_full['SAV ID'] == '283715236']

,SAV ID,SAV Name,Best Site ID,Best Site Customer Name,Best Site Sales Level 2 Name,Coverage,Contract Number,Covered Line Status,Contract Type,Contract Line End Quarter,...,SWSS Flag,Default Service List Price USD,Item Quantity,Annual Extended Contract Line List USD Amount,Annual Contract Line Net USD Amount,Annualized Extended Contract Line List USD Amount,Annualized Contract Line Net USD Amount,Contract Line List Price USD,Contract Line Net Price USD,Asset List Amount
110033,283715236,SBU_BR_DIST_OTHERS120,-9999,UNKNOWN,LATIN AMERICA,NOT COVERED,201041554,EXPIRED,ECMU,20191,...,Y,NaN,1.0,194.883929,122.44,184.505495,122.776374,184.00,122.44,NaN
110034,283715236,SBU_BR_DIST_OTHERS120,-9999,UNKNOWN,LATIN AMERICA,NOT COVERED,NaN,NaN,NaN,19009,...,N,NaN,1199.0,NaN,NaN,NaN,NaN,NaN,NaN,0.00
110035,283715236,SBU_BR_DIST_OTHERS120,-9999,UNKNOWN,LATIN AMERICA,NOT COVERED,NaN,NaN,NaN,19009,...,N,NaN,34.0,NaN,NaN,NaN,NaN,NaN,NaN,0.00
110036,283715236,SBU_BR_DIST_OTHERS120,-9999,UNKNOWN,LATIN AMERICA,NOT COVERED,NaN,NaN,NaN,19009,...,N,NaN,801.0,NaN,NaN,NaN,NaN,NaN,NaN,0.00
110037,283715236,SBU_BR_DIST_OTHERS120,-9999,UNKNOWN,LATIN AMERICA,NOT COVERED,NaN,NaN,NaN,19009,...,N,NaN,435.0,NaN,NaN,NaN,NaN,NaN,NaN,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218257,283715236,SBU_BR_DIST_OTHERS120,-9999,UNKNOWN,LATIN AMERICA,NOT COVERED,202146921,TERMINATED,SPAR1,20234,...,N,86.0,2.0,510.620000,0.00,118.259074,0.000000,510.62,0.00,1071.08
218258,283715236,SBU_BR_DIST_OTHERS120,-9999,UNKNOWN,LATIN AMERICA,COVERED,202146921,ACTIVE,SPAR1,20234,...,N,NaN,4.0,0.000000,0.00,0.000000,0.000000,0.00,0.00,517.88
218259,283715236,SBU_BR_DIST_OTHERS120,-9999,UNKNOWN,LATIN AMERICA,NOT COVERED,202146921,TERMINATED,SPAR1,20234,...,N,NaN,2.0,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00
218260,283715236,SBU_BR_DIST_OTHERS120,-9999,UNKNOWN,LATIN AMERICA,NOT COVERED,202146921,TERMINATED,SPAR1,20234,...,N,NaN,2.0,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00


In [5]:
#Loading necesary files
st_mapping = st_mapping[['Product SKU','L2NBD','L2SWT']]
IB = IB_full.copy()
lista_id = IB.iloc[:,0].unique().tolist()
IB

,SAV ID,SAV Name,Best Site ID,Best Site Customer Name,Best Site Sales Level 2 Name,Coverage,Contract Number,Covered Line Status,Contract Type,Contract Line End Quarter,...,SWSS Flag,Default Service List Price USD,Item Quantity,Annual Extended Contract Line List USD Amount,Annual Contract Line Net USD Amount,Annualized Extended Contract Line List USD Amount,Annualized Contract Line Net USD Amount,Contract Line List Price USD,Contract Line Net Price USD,Asset List Amount
0,222173994,BELIZE TELECOM,-9999,UNKNOWN,LATIN AMERICA,NOT COVERED,NaN,NaN,NaN,19009,...,N,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,512.01
1,222173994,BELIZE TELECOM,-9999,UNKNOWN,LATIN AMERICA,NOT COVERED,NaN,NaN,NaN,19009,...,N,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,2589.44
2,222173994,BELIZE TELECOM,-9999,UNKNOWN,LATIN AMERICA,NOT COVERED,NaN,NaN,NaN,19009,...,N,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,229.53
3,222173994,BELIZE TELECOM,-9999,UNKNOWN,LATIN AMERICA,NOT COVERED,NaN,NaN,NaN,19009,...,N,NaN,74.0,NaN,NaN,NaN,NaN,NaN,NaN,13064.70
4,222173994,BELIZE TELECOM,-9999,UNKNOWN,LATIN AMERICA,NOT COVERED,NaN,NaN,NaN,19009,...,N,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,423.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415803,283715230,SBU_PSR_MEX_CENTER_PSR105,-9999,UNKNOWN,LATIN AMERICA,COVERED,204218335,ACTIVE,PSUP,20251,...,N,165.0,1.0,479.85,86.276667,159.95,86.276667,479.85,258.83,2818.92
415804,283715230,SBU_PSR_MEX_CENTER_PSR105,-9999,UNKNOWN,LATIN AMERICA,NOT COVERED,NaN,NaN,NaN,19009,...,N,3636.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,89863.95
415805,283715230,SBU_PSR_MEX_CENTER_PSR105,-9999,UNKNOWN,LATIN AMERICA,NOT COVERED,NaN,NaN,NaN,19009,...,N,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,42889.88
415806,283715230,SBU_PSR_MEX_CENTER_PSR105,-9999,UNKNOWN,LATIN AMERICA,NOT COVERED,NaN,NaN,NaN,19009,...,N,2882.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,60536.00


In [6]:
%%time
#Loading necesary files
st_mapping = st_mapping[['Product SKU','L2NBD','L2SWT']]
IB_copy = IB_full.copy()
lista_id = IB_copy.iloc[:,0].unique().tolist()
lista_oppty =[]
lista_sav_id = []
for j in lista_id:
    IB=IB_copy[IB_copy.iloc[:,0] == j]
    #------------------------------------------------Calculating Success Tracks Column-------------------------------------------------
    #if product id is not on mapping, mark it as 'Not Elegible'
    merge1 = IB.merge(st_mapping, left_on='Product ID', right_on='Product SKU', how='left')
    merge1 = merge1.rename(columns={'Product SKU':'Success Tracks'})
    merge1['Success Tracks'] = merge1['Success Tracks'].fillna('Not Elegible')
    #-----------------------------------------------------------------------------------------------------------------
    # if contract type is in list_st mark it as 'Not Elegible'
    lista_st = ['L1NB3','L1NB5','L1NBD','L1SWT','L24H3','L24H5','L24HR','L2NB3','L2NB5','L2NBD','L2SWT']
    for i in merge1.index:
        if type(merge1.iloc[i,8]) == float:
            pass
        else:
            if merge1.iloc[i,8] in lista_st:
                merge1.iloc[i,33]=merge1.iloc[i,8].replace(merge1.iloc[i,8],'Not Elegible')
    #------------------------------------------------------------------------------------------------------------------
    #Else mark it as 'Elegible'
    for i in merge1.index:
        if merge1.iloc[i,33] != 'Not Elegible':
            merge1.iloc[i,33]=merge1.iloc[i,33].replace(merge1.iloc[i,33],'Elegible')

    #-----------------------------------------------Calculating SmartNet NBD ST Eligible-----------------------------------------------
    #If the product it's elegible use the annualized extended contract line list USD Amount, otherwise it's 0
    merge1['SmartNet NBD ST Eligible'] = merge1['Annualized Extended Contract Line List USD Amount']
    for i in merge1.index:
        if merge1.iloc[i,33] != 'Elegible':
            merge1.iloc[i,36] = 0
    #-----------------------------------------------Calculating DNA HW Appliance-----------------------------------------------
    #Check if 'DN2-HW-APL' or 'DN1-HW-APL' is in product ID and mark it as 1
    merge1['DNA HW Appliance'] = merge1['Product ID']
    for i in merge1.index:
        if 'DN2-HW-APL' in merge1.iloc[i,21] or 'DN1-HW-APL' in merge1.iloc[i,21]:
            merge1.iloc[i,37] = merge1.iloc[i,37].replace(merge1.iloc[i,37], '1')
    # If not mark it as 0
    for i in merge1.index:
        if merge1.iloc[i,37] != '1':
            merge1.iloc[i,37] = '0'

    #-----------------------------------------------Calculating SmartNet ST Not Covered-----------------------------------------------
    # If it's not covered and 1 in DNA HW Appliance get the annualized extended contract line list USD Amount
    merge1['SmartNet ST Not Covered'] = 0
    for i in merge1.index:
        if merge1.iloc[i,5] == 'NOT COVERED' and merge1.iloc[i,37] == '1' :
            merge1.iloc[i,38] = merge1.iloc[i,28]

    #-----------------------------------------------Calculating ST Estimated List Price-----------------------------------------------
    #merge1['ST Estimated List Price'] = (merge1['L2NBD'] + merge1['L2SWT'])*merge1['Item Quantity']
    merge1['ST Estimated List Price'] = merge1['L2NBD']*merge1['Item Quantity']

    #------------------------------------------------------Filtering------------------------------------------------------------------
    merge1 = merge1[merge1['Coverage'] == 'COVERED']
    merge1 = merge1[merge1['Success Tracks'] == 'Elegible']

    #----------------------------------------Calculating Total ST Oppty KPI------------------------------------------------------------

    sn_nbd_st_e = merge1['SmartNet NBD ST Eligible'].sum()
    sn_st_not_covered = merge1['SmartNet ST Not Covered'].sum()
    st_e_list_price = merge1['ST Estimated List Price'].sum()
    ST_Oppty= st_e_list_price - sn_st_not_covered - sn_nbd_st_e

    if len(merge1) == 0:
        lista_oppty.append('N/A')
        lista_sav_id.append(j)
        print('Account ID: ',j,'ST_Oppty: ', 'N/A')
        
        
    if ST_Oppty < 0:
        
        merge1 = IB.merge(st_mapping, left_on='Product ID', right_on='Product SKU', how='left')
        merge1 = merge1.rename(columns={'Product SKU':'Success Tracks'})
        merge1['Success Tracks'] = merge1['Success Tracks'].fillna('Not Elegible')
        #-----------------------------------------------------------------------------------------------------------------
        # if contract type is in list_st mark it as 'Not Elegible'
        lista_st = ['L1NB3','L1NB5','L1NBD','L1SWT','L24H3','L24H5','L24HR','L2NB3','L2NB5','L2NBD','L2SWT']
        for i in merge1.index:
            if type(merge1.iloc[i,8]) == float:
                pass
            else:
                if merge1.iloc[i,8] in lista_st:
                    merge1.iloc[i,33]=merge1.iloc[i,8].replace(merge1.iloc[i,8],'Not Elegible')
        #------------------------------------------------------------------------------------------------------------------
        #Else mark it as 'Elegible'
        for i in merge1.index:
            if merge1.iloc[i,33] != 'Not Elegible':
                merge1.iloc[i,33]=merge1.iloc[i,33].replace(merge1.iloc[i,33],'Elegible')


        merge1['SmartNet NBD ST Eligible'] = merge1['Default Service List Price USD']
        for i in merge1.index:
            if merge1.iloc[i,33] != 'Elegible':
                merge1.iloc[i,36] = 0
        #-----------------------------------------------Calculating DNA HW Appliance-----------------------------------------------
        #Check if 'DN2-HW-APL' or 'DN1-HW-APL' is in product ID and mark it as 1
        merge1['DNA HW Appliance'] = merge1['Product ID']
        for i in merge1.index:
            if 'DN2-HW-APL' in merge1.iloc[i,21] or 'DN1-HW-APL' in merge1.iloc[i,21]:
                merge1.iloc[i,37] = merge1.iloc[i,37].replace(merge1.iloc[i,37], '1')
        # If not mark it as 0
        for i in merge1.index:
            if merge1.iloc[i,37] != '1':
                merge1.iloc[i,37] = '0'

        #-----------------------------------------------Calculating SmartNet ST Not Covered-----------------------------------------------
        # If it's not covered and 1 in DNA HW Appliance get the annualized extended contract line list USD Amount
        merge1['SmartNet ST Not Covered'] = 0
        for i in merge1.index:
            if merge1.iloc[i,5] == 'NOT COVERED' and merge1.iloc[i,37] == '1' :
                merge1.iloc[i,38] = merge1.iloc[i,24]

        #-----------------------------------------------Calculating ST Estimated List Price-----------------------------------------------
        #merge1['ST Estimated List Price'] = (merge1['L2NBD'] + merge1['L2SWT'])*merge1['Item Quantity']
        merge1['ST Estimated List Price'] = merge1['L2NBD']*merge1['Item Quantity']

        #------------------------------------------------------Filtering------------------------------------------------------------------
        merge1 = merge1[merge1['Coverage'] == 'COVERED']
        merge1 = merge1[merge1['Success Tracks'] == 'Elegible']

        #----------------------------------------Calculating Total ST Oppty KPI------------------------------------------------------------

        sn_nbd_st_e = merge1['SmartNet NBD ST Eligible'].sum()
        sn_st_not_covered = merge1['SmartNet ST Not Covered'].sum()
        st_e_list_price = merge1['ST Estimated List Price'].sum()
        ST_Oppty= st_e_list_price - sn_st_not_covered - sn_nbd_st_e
        lista_oppty.append(round(ST_Oppty/1000, 1))
        lista_sav_id.append(j)
        print('Account ID: ',j,'ST_Oppty: ', round(ST_Oppty/1000, 1), 'Estimated list price: ', st_e_list_price, 'Originally Negative')
    else:
        lista_oppty.append(round(ST_Oppty/1000, 1))
        lista_sav_id.append(j)
        print('Account ID: ',j,'ST_Oppty: ', round(ST_Oppty/1000, 1), 'Estimated list price: ', st_e_list_price)


Account ID:  222173994 ST_Oppty:  32.5 Estimated list price:  98831.29000000001
Account ID:  230366718 ST_Oppty:  581.9 Estimated list price:  1121225.82
Account ID:  203840606 ST_Oppty:  615.8 Estimated list price:  1015879.34
Account ID:  207593001 ST_Oppty:  73.5 Estimated list price:  237063.9
Account ID:  283468065 ST_Oppty:  23.7 Estimated list price:  79735.88
Account ID:  203772167 ST_Oppty:  282.7 Estimated list price:  574074.92
Account ID:  203772245 ST_Oppty:  98.8 Estimated list price:  152665.31
Account ID:  203711644 ST_Oppty:  229.5 Estimated list price:  436248.95999999996
Account ID:  203849413 ST_Oppty:  324.9 Estimated list price:  534043.56
Account ID:  203701565 ST_Oppty:  512.7 Estimated list price:  761234.36
Account ID:  283536694 ST_Oppty:  400.8 Estimated list price:  979038.4199999999
Account ID:  203743221 ST_Oppty:  187.0 Estimated list price:  536276.16
Account ID:  233445185 ST_Oppty:  96.2 Estimated list price:  255020.72
Account ID:  203785620 ST_Oppty

In [7]:
df_st_oppty = pd.DataFrame()
df_st_oppty['ID'] = lista_sav_id
df_st_oppty['ST_Oppty'] = lista_oppty
df_st_oppty


,ID,ST_Oppty
0,222173994,32.5
1,230366718,581.9
2,203840606,615.8
3,207593001,73.5
4,283468065,23.7
...,...,...
1334,283714968,1022.0
1335,203797997,22797.1
1336,283715236,4357.3
1337,283714970,2029.1


In [8]:

df_st_oppty.to_csv('ST_Oppty_5-2-2022.csv')